## WANDS

In [1]:
!git clone https://github.com/wayfair/WANDS.git

Cloning into 'WANDS'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 40 (delta 6), reused 3 (delta 3), pack-reused 21 (from 1)
Receiving objects: 100% (40/40), 33.32 MiB | 11.04 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [6]:
!pip install pandas


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [7]:
import pandas as pd

In [8]:
query_df = pd.read_csv("WANDS/dataset/query.csv", sep='\t')
query_df

,query_id,query,query_class
0,0,salon chair,Massage Chairs
1,1,smart coffee table,Coffee & Cocktail Tables
2,2,dinosaur,Kids Wall Décor
3,3,turquoise pillows,Accent Pillows
4,4,chair and a half recliner,Recliners
...,...,...,...
475,483,rustic twig,Faux Plants and Trees
476,484,nespresso vertuo next premium by breville with...,Espresso Machines
477,485,pedistole sink,Kitchen Sinks
478,486,54 in bench cushion,Furniture Cushions


## Tests

In [10]:
import requests

In [25]:
QUEPID_TOKEN = '79d8d8c2106e4123b999b3d369adcab970622fafe1ca4ad75578bb730cf26155'

### Create team

In [31]:
team = requests.post(
    'http://localhost:8081/api/teams/', 
    headers = {
        "Authorization": f"Bearer {QUEPID_TOKEN}"
    },
    json={
        "name": "wands"
    }   
)

team = team.json()

{'id': 3,
 'name': 'wands',
 'created_at': '2025-07-06T15:16:21.242Z',
 'updated_at': '2025-07-06T15:16:21.242Z'}

### Create team

In [53]:
endpoint = requests.post(
    'http://localhost:8081/api/search_endpoints/', 
    headers = {
        "Authorization": f"Bearer {QUEPID_TOKEN}"
    },
    json={
        "name": "wands",
        "endpoint_url": "http://localhost:9200/",
        "search_engine": "es",
        "api_method": "POST",
        "proxy_requests": 1,   
    }   
)

endpoint = endpoint.json()

In [54]:
endpoint

{'id': 8,
 'name': 'wands',
 'owner': 1,
 'search_engine': 'es',
 'endpoint_url': 'http://localhost:9200/',
 'api_method': 'POST',
 'custom_headers': None,
 'archived': 0,
 'created_at': '2025-07-06T16:37:25.484Z',
 'updated_at': '2025-07-06T16:37:25.484Z',
 'basic_auth_credential': None,
 'mapper_code': None,
 'proxy_requests': None,
 'options': None}

### Create case

In [56]:
case = requests.post(
    'http://localhost:8081/api/case/', 
    headers = {
        "Authorization": f"Bearer {QUEPID_TOKEN}"
    },
    json={
        "name": "string 222",
        "scorer_id": 5,
        "book_id": 0,
        "search_endpoint_id": endpoint.get('id')
    }   
)

print(case, case.content)

case = case.json()

<Response [200]> b'{"id": 28, "case_name": "string 222", "last_try_number": 1, "owner": 1, "archived": 0, "scorer_id": 5, "created_at": "2025-07-06T16:38:23.849Z", "updated_at": "2025-07-06T16:38:23.849Z", "book_id": null, "public": null, "options": null, "nightly": null}'


In [47]:
case

<Response [502]>

### Create queries

In [57]:
query = requests.post(
    f'http://localhost:8081/api/query/{case.get("id")}/', 
    headers = {
        "Authorization": f"Bearer {QUEPID_TOKEN}"
    },
    json={
        "query_text": "string 222"
    }   
)

query = query.json()
query

{'query_options': {},
 'id': 3,
 'arranged_next': None,
 'arranged_at': None,
 'query_text': 'string 222',
 'notes': '',
 'case': 28,
 'created_at': '2025-07-06T16:38:26.627Z',
 'updated_at': '2025-07-06T16:38:26.627Z',
 'information_need': ''}